In [17]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import datetime as dt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
f_g_df = pd.read_csv('/content/drive/MyDrive/ds_thanseela/csv_files/fear_greed_index.csv')
tradess = pd.read_csv('/content/drive/MyDrive/ds_thanseela/csv_files/historical_data.csv')

print("Fear & Greed dataset shape:", fear_greed.shape)
print("trades dataset shape:", tradess.shape)

fear_greed.head(), tradess.head()


Fear & Greed dataset shape: (2644, 4)
trades dataset shape: (211224, 16)


(    timestamp  value classification        date
 0  1517463000     30           Fear  01-02-2018
 1  1517549400     15   Extreme Fear  02-02-2018
 2  1517635800     40           Fear  03-02-2018
 3  1517722200     24   Extreme Fear  04-02-2018
 4  1517808600     11   Extreme Fear  05-02-2018,
                                       Account  Coin  Execution Price  \
 0  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9769   
 1  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9800   
 2  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9855   
 3  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9874   
 4  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9894   
 
    Size Tokens  Size USD Side     Timestamp IST  Start Position Direction  \
 0       986.87   7872.16  BUY  02-12-2024 22:50        0.000000       Buy   
 1        16.00    127.68  BUY  02-12-2024 22:50      986.524596       Buy   
 2       144.09   1150.63  BUY 

In [19]:
# --- Sentiment data (f_g_df) ---
f_g_df['timestamp'] = pd.to_datetime(f_g_df['timestamp'], unit='s', errors='coerce')
f_g_df['date'] = f_g_df['timestamp'].dt.date  # keep only date

# --- Trading data (trades) ---
trades['Timestamp'] = pd.to_datetime(trades['Timestamp'], unit='ms', errors='coerce')
trades['trades_date'] = trades['Timestamp'].dt.date  # extract only date

# ✅ Check values
print(f_g_df[['timestamp', 'date', 'classification']].head())
print(trades[['Timestamp', 'trades_date', 'Closed PnL']].head())


            timestamp        date classification
0 2018-02-01 05:30:00  2018-02-01           Fear
1 2018-02-02 05:30:00  2018-02-02   Extreme Fear
2 2018-02-03 05:30:00  2018-02-03           Fear
3 2018-02-04 05:30:00  2018-02-04   Extreme Fear
4 2018-02-05 05:30:00  2018-02-05   Extreme Fear
            Timestamp trades_date  Closed PnL
0 2024-10-27 03:33:20  2024-10-27         0.0
1 2024-10-27 03:33:20  2024-10-27         0.0
2 2024-10-27 03:33:20  2024-10-27         0.0
3 2024-10-27 03:33:20  2024-10-27         0.0
4 2024-10-27 03:33:20  2024-10-27         0.0


In [20]:
f_g_df = f_g_df.rename(columns={
    'classification': 'Classification',
    'value': 'Sentiment_Value'
})

trades = trades.rename(columns={
    'Execution Price': 'Execution_Price',
    'Size Tokens': 'Size_Tokens',
    'Size USD': 'Size_USD',
    'Closed PnL': 'Closed_PnL'
})


In [21]:
merged = pd.merge(
    trades,
    f_g_df[['date', 'Classification', 'Sentiment_Value']],
    left_on='trades_date',
    right_on='date',
    how='left'
)

print("✅ Merged dataset shape:", merged.shape)
print(merged[['trades_date', 'Classification', 'Closed_PnL', 'Size_USD']].head())


✅ Merged dataset shape: (211224, 20)
  trades_date Classification  Closed_PnL  Size_USD
0  2024-10-27          Greed         0.0   7872.16
1  2024-10-27          Greed         0.0    127.68
2  2024-10-27          Greed         0.0   1150.63
3  2024-10-27          Greed         0.0   1142.04
4  2024-10-27          Greed         0.0     69.75


In [22]:
sentiment_summary = merged.groupby('Classification').agg({
    'Closed_PnL': ['mean', 'median'],
    'Size_USD': 'mean',
    'Execution_Price': 'mean',
    'Sentiment_Value': 'mean'
}).reset_index()

print("📊 Summary by Market Sentiment:\n")
print(sentiment_summary)



📊 Summary by Market Sentiment:

  Classification Closed_PnL            Size_USD Execution_Price  \
                       mean median         mean            mean   
0  Extreme Greed  25.418772    0.0  5660.265764     9771.897315   
1           Fear  50.047622    0.0  5259.977837    11102.057978   
2          Greed  87.894859    0.0  3182.883845     5513.715386   
3        Neutral  22.229713    0.0  3058.848110     2663.132338   

  Sentiment_Value  
             mean  
0       84.000000  
1       44.000000  
2       73.854777  
3       50.000000  


In [23]:

plt.figure(figsize=(8,5))
sns.barplot(data=sentiment_summary, x='Classification', y=('Closed_PnL','mean'))
plt.title('Average tradesr PnL vs Market Sentiment')
plt.xlabel('Market Sentiment')
plt.ylabel('Average Closed PnL')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [25]:
# Select only numeric columns for correlation
numeric_cols = ['Sentiment_Value', 'Closed_PnL', 'Size_USD', 'Execution_Price']
corr_matrix = merged[numeric_cols].corr()

print("📈 Correlation Matrix:\n")
print(corr_matrix)

# --- Visualization ---
plt.figure(figsize=(6,4))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm')
plt.title("Correlation Between Sentiment Value and Trader Metrics")
plt.tight_layout()
plt.show()

📈 Correlation Matrix:

                 Sentiment_Value  Closed_PnL  Size_USD  Execution_Price
Sentiment_Value         1.000000    0.011132 -0.024110        -0.070793
Closed_PnL              0.011132    1.000000  0.123589        -0.006505
Size_USD               -0.024110    0.123589  1.000000         0.189855
Execution_Price        -0.070793   -0.006505  0.189855         1.000000


In [26]:
import statsmodels.api as sm

# Drop missing values
reg_data = merged[['Sentiment_Value', 'Closed_PnL']].dropna()

# Define X (independent) and y (dependent)
X = reg_data['Sentiment_Value']
y = reg_data['Closed_PnL']

# Add a constant term for intercept
X = sm.add_constant(X)

# Fit regression model
model = sm.OLS(y, X).fit()

print("📊 Regression Summary:")
print(model.summary())


📊 Regression Summary:
                            OLS Regression Results                            
Dep. Variable:             Closed_PnL   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     22.84
Date:                Fri, 31 Oct 2025   Prob (F-statistic):           1.77e-06
Time:                        04:55:52   Log-Likelihood:            -1.5225e+06
No. Observations:              184263   AIC:                         3.045e+06
Df Residuals:                  184261   BIC:                         3.045e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              1